# Fine-tuning a classifier using bounding box data from a 3LC Table

In this tutorial, we will fine-tune a classifier using bounding box data from a 3LC `Table`.

We will load the COCO128 table from an earlier notebook and use it to create a
`torch.utils.Dataset` of bounding box crops. These cropped images will be used to
fine-tune a classifier. In a later tutorial, we will use this trained model to
generate embeddings and predicted labels.

## Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import tlc

from tlc_tools.augment_bbs.finetune_on_crops import train_model
from tlc_tools.common import infer_torch_device
from tlc_tools.split import split_table

## Project setup

In [3]:
EPOCHS = 10
MODEL_CHECKPOINT = "../../../transient_data/seg_classifier.pth"
MODEL_NAME = "efficientnet_b0"
INCLUDE_BACKGROUND = False

## Set device

In [4]:
DEVICE = infer_torch_device()
print(f"Using device: {DEVICE}")

Using device: cuda


## Load input Table

We will reuse the table created in the notebook [create-table-from-coco.ipynb](../../1-create-tables/create-table-from-coco.ipynb).

In [5]:
input_table = tlc.Table.from_names(
    "initial-segmentation",
    "COCO128",
    "3LC Tutorials",
)

3lc: Loaded project alias configuration from C:/Users/gudbrand/AppData/Local/3LC/3LC/projects/3LC Tutorials/default_aliases.3lc.yaml


## Split the Table

In [6]:
# Create splits for training and validation
splits = split_table(input_table, {"train": 0.8, "val": 0.2})

train_table = splits["train"]
val_table = splits["val"]

print(f"Using table {train_table} for training")
print(f"Using table {val_table} for validation")

Using table EditedTable(project_name="3LC Tutorials", dataset_name="COCO128", name="train_0003", row_count=103) for training
Using table EditedTable(project_name="3LC Tutorials", dataset_name="COCO128", name="val_0003", row_count=25) for validation


## Train model

In [7]:
from tlc_tools.common import InstanceConfig


model, checkpoint_path = train_model(
    train_table_url=train_table.url,
    val_table_url=val_table.url,
    model_checkpoint=MODEL_CHECKPOINT,
    epochs=1,
    include_background=INCLUDE_BACKGROUND,
    num_workers=0,  # Use more workers in a non-notebook environment
    instance_config=InstanceConfig(
        instance_column="segmentations",
        instance_type="segmentation_masks",
        label_column_path="segmentations.instance_properties.label",
    ),
)

Using device: cuda
Instance configuration for training:
  Instance column: segmentations
  Instance type: segmentation_masks
  Label column path: instance_properties.label


ValueError: Label map not found in table at path: instance_properties.label